In [ ]:
__author__ = 'David Herrera <david.herrera@noirlab.edu>, and the Astro Data Lab Team <datalab@noirlab.edu>'
__version__ = '20230515' # yyyymmdd
__datasets__ = ['','']
__keywords__ = ['extragalactic','galaxies','joint query','spectroscopic redshift','3d plot']

### Table of contents
* [Goals & Summary](#goals)
* [Disclaimer & attribution](#attribution)
* [Imports & setup](#import)
* [Joint Query of LS and SDSS catalogs](#query)
* [Plot Results](#plots)
* [Exercise](#exercise)
* [3D plot (RA,DEC and z)](3d_plot)

<a class="anchor" id="goals"></a>
# Goals
* Reproduce plots from the DESI LIS paper with the latest datasets

# Summary

In this Notebook, 

In [ ]:
# std lib
from getpass import getpass

# 3rd party
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import colors
from scipy.stats import binned_statistic_2d
%matplotlib inline
from astropy.table import Table
from astropy.cosmology import Planck18 as cosmo
import plotly
import plotly.graph_objs as go
import pandas as pd
plotly.offline.init_notebook_mode()

# Data Lab
from dl import queryClient as qc
from dl import authClient as ac

print('Done importing')

In [ ]:
# Uncomment the next 3 lines in case authentication is needed:
#token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
#if not ac.isValidToken(token):
#    raise Exception('Token is not valid. Please check your usename/password and execute this cell again.')

In [ ]:
#Fig. 1 - All sky map:
query = ("""SELECT avg(ra) as ra0, avg(dec) as dec0, nest4096, count(nest4096) as n
FROM ls_dr7.tractor""")
print(query)

.  

## Fig. 15
.  
.  



In [ ]:
# Fig. 15
query15a = ("""SELECT 
ra as d6ra,dec as d6dec , nexp_g as d6neg, nexp_r as d6ner, nexp_z as d6nez,
nexphist_g_1 as d6ng1,nexphist_g_2 as d6ng2,nexphist_g_3 as d6ng3,nexphist_g_4 as d6ng4,nexphist_g_5 as d6ng5,nexphist_g_6 as d6ng6,
nexphist_r_1 as d6nr1,nexphist_r_2 as d6nr2,nexphist_r_3 as d6nr3,nexphist_r_4 as d6nr4,nexphist_r_5 as d6nr5,nexphist_r_6 as d6nr6,
nexphist_z_1 as d6nz1,nexphist_z_2 as d6nz2,nexphist_z_3 as d6nz3,nexphist_z_4 as d6nz4,nexphist_z_5 as d6nz5,nexphist_z_6 as d6nz6,
psfdepth_g as d6psfdg,psfdepth_r as d6psfdr,psfdepth_z as d6psfdz,
nobjs
FROM ls_dr6.bricks_dr6 as d6b""")
#print(query15a)
query15b = ("""SELECT 
ra as d7ra,dec as d7dec , nexp_g as d7neg, nexp_r as d7ner, nexp_z as d7nez,
nexphist_g_1 as d7ng1,nexphist_g_2 as d7ng2,nexphist_g_3 as d7ng3,nexphist_g_4 as d7ng4,nexphist_g_5 as d7ng5,nexphist_g_6 as d7ng6,
nexphist_r_1 as d7nr1,nexphist_r_2 as d7nr2,nexphist_r_3 as d7nr3,nexphist_r_4 as d7nr4,nexphist_r_5 as d7nr5,nexphist_r_6 as d7nr6,
nexphist_z_1 as d7nz1,nexphist_z_2 as d7nz2,nexphist_z_3 as d7nz3,nexphist_z_4 as d7nz4,nexphist_z_5 as d7nz5,nexphist_z_6 as d7nz6,
psfdepth_g as d7psfdg,psfdepth_r as d7psfdr,psfdepth_z as d7psfdz,
nobjs
FROM ls_dr7.bricks_dr7 as d7b""")
#print(query15b)
query15c = ("""SELECT
ra as d9ra,dec as d9dec , nexp_g as d9neg, nexp_r as d9ner, nexp_z as d9nez,
nexphist_g_1 as d9ng1,nexphist_g_2 as d9ng2,nexphist_g_3 as d9ng3,nexphist_g_4 as d9ng4,nexphist_g_5 as d9ng5,nexphist_g_6 as d9ng6,
nexphist_r_1 as d9nr1,nexphist_r_2 as d9nr2,nexphist_r_3 as d9nr3,nexphist_r_4 as d9nr4,nexphist_r_5 as d9nr5,nexphist_r_6 as d9nr6,
nexphist_z_1 as d9nz1,nexphist_z_2 as d9nz2,nexphist_z_3 as d9nz3,nexphist_z_4 as d9nz4,nexphist_z_5 as d9nz5,nexphist_z_6 as d9nz6,
psfdepth_g as d9psfdg,psfdepth_r as d9psfdr,psfdepth_z as d9psfdz,
nobjs
FROM ls_dr9.bricks_s as d9b""")
print(query15c)

In [ ]:
# Fetch the DESI LIS data from ls_dr6.bricks_dr6 (d6) and from ls_dr7.bricks_dr7 (d7) 
d6 = qc.query(sql=query15a, fmt='pandas')
d7 = qc.query(sql=query15b, fmt='pandas')
d9 = qc.query(sql=query15c, fmt='pandas')

#print(d6[:5])
#print(d7[:5])
print(d9[:5])

In [ ]:
# Required higher
nmin = 1000
#Requires 90% of the image are to contain  >= 3 exposures:
npix = 0.9*900.*900. 

In [ ]:
#Add all histograms of pixels per brick per band g,r,z
total_ng6 = d6['d6ng1'] + d6['d6ng2'] + d6['d6ng3'] + d6['d6ng4'] + d6['d6ng5'] + d6['d6ng6']
total_nr6 = d6['d6nr1'] + d6['d6nr2'] + d6['d6nr3'] + d6['d6nr4'] + d6['d6nr5'] + d6['d6nr6']
total_nz6 = d6['d6nz1'] + d6['d6nz2'] + d6['d6nz3'] + d6['d6nz4'] + d6['d6nz5'] + d6['d6nz6']
total_ng7 = d7['d7ng1'] + d7['d7ng2'] + d7['d7ng3'] + d7['d7ng4'] + d7['d7ng5'] + d7['d7ng6']
total_nr7 = d7['d7nr1'] + d7['d7nr2'] + d7['d7nr3'] + d7['d7nr4'] + d7['d7nr5'] + d7['d7nr6']
total_nz7 = d7['d7nz1'] + d7['d7nz2'] + d7['d7nz3'] + d7['d7nz4'] + d7['d7nz5'] + d7['d7nz6']

#Add all histograms of pixels per brick per band g,r,z with nexp >= 3
ge3_ng6 = d6['d6ng4'] + d6['d6ng5'] + d6['d6ng6']
ge3_nr6 = d6['d6nr4'] + d6['d6nr5'] + d6['d6nr6']
ge3_nz6 = d6['d6nz4'] + d6['d6nz5'] + d6['d6nz6']
ge3_ng7 = d7['d7ng4'] + d7['d7ng5'] + d7['d7ng6'] 
ge3_nr7 = d7['d7nr4'] + d7['d7nr5'] + d7['d7nr6'] 
ge3_nz7 = d7['d7nz4'] + d7['d7nz5'] + d7['d7nz6']

#Normalizing to 1
fraction_ng6 = ge3_ng6 / total_ng6
fraction_nr6 = ge3_nr6 / total_nr6
fraction_nz6 = ge3_nz6 / total_nz6
fraction_ng7 = ge3_ng7 / total_ng7
fraction_nr7 = ge3_nr7 / total_nr7
fraction_nz7 = ge3_nz7 / total_nz7

# Conditions to consider values
cond6g = (d6['d6neg'] >= 3)  & (d6['nobjs'] >= nmin)  & (total_ng6 > npix) &  (fraction_ng6 >= 0.9)
cond6r = (d6['d6ner'] >= 3)  & (d6['nobjs'] >= nmin)  & (total_nr6 > npix) &  (fraction_nr6 >= 0.9)
cond6z = (d6['d6nez'] >= 3)  & (d6['nobjs'] >= nmin)  & (total_nz6 > npix) &  (fraction_nz6 >= 0.9)
cond7g = (d7['d7neg'] >= 3)  & (d7['nobjs'] >= nmin)  & (total_ng7 > npix) &  (fraction_ng7 >= 0.9)
cond7r = (d7['d7ner'] >= 3)  & (d7['nobjs'] >= nmin)  & (total_nr7 > npix) &  (fraction_nr7 >= 0.9)
cond7z = (d7['d7nez'] >= 3)  & (d7['nobjs'] >= nmin)  & (total_nz7 > npix) &  (fraction_nz7 >= 0.9)

#print(cond6g[20000:20040])

In [ ]:
#We stablish the bin range and size 
my_bins = np.arange(22.5,26.0001,0.02)
#my_bins

In [ ]:
cpc_g6,bins = np.histogram(d6['d6psfdg'][cond6g], bins=my_bins,range = (22.5, 26.0))
cpc_r6,bins = np.histogram(d6['d6psfdr'][cond6r], bins=my_bins,range = (22.5, 26.0))
cpc_z6,bins = np.histogram(d6['d6psfdz'][cond6z], bins=my_bins,range = (22.5, 26.0))
cpc_g7,bins = np.histogram(d7['d7psfdg'][cond7g], bins=my_bins,range = (22.5, 26.0))
cpc_r7,bins = np.histogram(d7['d7psfdr'][cond7r], bins=my_bins,range = (22.5, 26.0))
cpc_z7,bins = np.histogram(d7['d7psfdz'][cond7z], bins=my_bins,range = (22.5, 26.0))

#print(cpc_g6)
#print(cpc_g7)
#print(cpc_z6)

In [ ]:
centers = (bins[0:-1]+bins[1:])/2

In [ ]:
plt.bar(centers,cpc_g6/maxg6)

In [ ]:
# Calculate the cumulative sum, maximum and the fraction of each band

cpcf_g1 = np.cumsum(cpc_g6)
max_g6 = cpcf_g1.max()
cpcf_g1 = cpcf_g1/max_g6
cpcf_r1 = np.cumsum(cpc_r6)
max_r6 = cpcf_r1.max()
cpcf_r1 = cpcf_r1/max_r6
cpcf_z1 = np.cumsum(cpc_z6)
max_z6 = cpcf_z1.max()
cpcf_z1 = cpcf_z1/max_z6

cpcf_g2 = np.cumsum(cpc_g7)
max_g7 = cpcf_g2.max()
cpcf_g2 = cpcf_g2/max_g7

cpcf_r2 = np.cumsum(cpc_r7)
max_r7 = cpcf_r2.max()
cpcf_r2 = cpcf_r2/max_r7

cpcf_z2 = np.cumsum(cpc_z7)
max_z7 = cpcf_z2.max()
cpcf_z2 = cpcf_z2/max_z7

In [ ]:
font = {'family' : 'monospace',
        'weight' : 'bold',
        'size'   : 16}

matplotlib.rc('font', **font)
#fig, ax = plt.subplots()
                       
plt.figure(figsize=(9,8))
plt.xlim(22.5,26.0)
plt.ylim(0,1.)
plt.xlabel('AB Magnitude', fontsize = 20)
plt.ylabel('Cumulative fraction', fontsize = 20)
#plt.Axes.tick_params(axis='both', direction = 'in')
#ax.tick_params(axis='both', direction = 'in')
plt.plot(centers,cpcf_g2, c='blue', label='DR7 g', lw=0.8)
plt.plot(centers,cpcf_g1, c='blue', ls='dashed', label='DR6 g', lw=0.8)
plt.plot(centers,cpcf_r2, c='red', label='DR7 r', lw=0.8)
plt.plot(centers,cpcf_r1, c='red', ls='dashed', label='DR6 r', lw=0.8)
plt.plot(centers,cpcf_z2, c='purple', label='DR7 z', lw=0.8)
plt.plot(centers,cpcf_z1, c='purple', ls='dashed', label='DR6 z', lw=0.8)

plt.legend(loc=7,frameon=False)
plt.show()

In [ ]:
total_ng9 = d9['d9ng1'] + d9['d9ng2'] + d9['d9ng3'] + d9['d9ng4'] + d9['d9ng5'] + d9['d9ng6']
total_nr9 = d9['d9nr1'] + d9['d9nr2'] + d9['d9nr3'] + d9['d9nr4'] + d9['d9nr5'] + d9['d9nr6']
total_nz9 = d9['d9nz1'] + d9['d9nz2'] + d9['d9nz3'] + d9['d9nz4'] + d9['d9nz5'] + d9['d9nz6']

ge3_ng9 = (d9['d9ng3'] + d9['d9ng4'] + d9['d9ng5'] + d9['d9ng6'])
ge3_nr9 = (d9['d9nr3'] + d9['d9nr4'] + d9['d9nr5'] + d9['d9nr6'])
ge3_nz9 = (d9['d9nz3'] + d9['d9nz4'] + d9['d9nz5'] + d9['d9nz6'])

fraction_ng9 = ge3_ng9 / total_ng9
fraction_nr9 = ge3_nr9 / total_nr9
fraction_nz9 = ge3_nz9 / total_nz9

cond9g = (d9['d9neg'] >= 3)  & (d9['nobjs'] >= nmin)  & (total_ng9 > npix) &  (fraction_ng9 >= 0.9)
cond9r = (d9['d9ner'] >= 3)  & (d9['nobjs'] >= nmin)  & (total_nr9 > npix) &  (fraction_nr9 >= 0.9)
cond9z = (d9['d9nez'] >= 3)  & (d9['nobjs'] >= nmin)  & (total_nz9 > npix) &  (fraction_nz9 >= 0.9)

cpc_g9,bins = np.histogram(d9['d9psfdg'][cond9g], bins=my_bins,range = (22.5, 29.0))
cpc_r9,bins = np.histogram(d9['d9psfdr'][cond9r], bins=my_bins,range = (22.5, 29.0))
cpc_z9,bins = np.histogram(d9['d9psfdz'][cond9z], bins=my_bins,range = (22.5, 29.0))
#print(cpc_g9)
#print(cpc_r9)
#print(cpc_z9)

cpcf_g3 = np.cumsum(cpc_g9)
max_g9 = cpcf_g3.max()
cpcf_g3 = cpcf_g3/max_g9
cpcf_r3 = np.cumsum(cpc_r9)
max_r9 = cpcf_r3.max()
cpcf_r3 = cpcf_r3/max_r9
cpcf_z3 = np.cumsum(cpc_z9)
max_z9 = cpcf_z3.max()
cpcf_z3 = cpcf_z3/max_z9

In [ ]:
plt.figure(figsize=(9,8))
plt.xlim(22.5,26.0)
plt.ylim(0,1.)
plt.xlabel('AB Magnitude', fontsize = 20)
plt.ylabel('Cumulative fraction', fontsize = 20)

plt.plot(centers,cpcf_g3, c='blue', label='DR9 g', lw=0.8)
plt.plot(centers,cpcf_r3, c='red', label='DR9 r', lw=0.8)
plt.plot(centers,cpcf_z3, c='purple', label='DR9 z', lw=0.8)

In [ ]:
plt.figure(figsize=(9,8))
plt.xlim(22.5,26.0)
plt.ylim(0,1.)
plt.xlabel('AB Magnitude', fontsize = 20)
plt.ylabel('Cumulative fraction', fontsize = 20)

plt.plot(centers,cpcf_g2, c='blue', label='DR7 g', lw=0.8)
plt.plot(centers,cpcf_g1, c='blue', ls='dashed', label='DR6 g', lw=0.8)
plt.plot(centers,cpcf_r2, c='red', label='DR7 r', lw=0.8)
plt.plot(centers,cpcf_r1, c='red', ls='dashed', label='DR6 r', lw=0.8)
plt.plot(centers,cpcf_z2, c='purple', label='DR7 z', lw=0.8)
plt.plot(centers,cpcf_z1, c='purple', ls='dashed', label='DR6 z', lw=0.8)
plt.plot(centers,cpcf_g3, c='blue', label='DR9 g', ls='solid', lw=2)
plt.plot(centers,cpcf_r3, c='red', label='DR9 r', ls='solid',lw=2)
plt.plot(centers,cpcf_z3, c='purple', label='DR9 z',ls='solid', lw=2)

plt.legend(loc=7,frameon=False)
plt.show()

.  
## Fig. 8

.  
.  

In [ ]:
# Fig. 8

query_fig8 = ("""
select l.mag_w1, l.mag_w2, l.dered_mag_w1, l.dered_mag_w2, a.w1mpro +
2.699 as w1mpro, a.w2mpro + 3.339 as w2mpro
from ls_dr7.tractor as l, allwise.source as a
where q3c_radial_query(a.ra,a.dec,l.ra,l.dec,1/3600.) 
and l.random_id between 45. and 45.23 limit 200000
""")
print (query_fig8)

In [ ]:
# Fetch the W1 mag from ls_dr17.tractor and from allwise.source
#w1_awls = qc.query(sql=query_fig8, fmt='csv')

#data_awls = Table.read(w1_awls, format='csv')

#data_awls[:10]

In [ ]:
#len(data_awls)
# Fetch the W1 mag from ls_dr17.tractor and from allwise.source
w1_awls = qc.query(sql=query_fig8, fmt='pandas')

In [ ]:
w1_awls 

In [ ]:
w1_awls.index[np.isinf(w1_awls).any(1)]

In [ ]:
foo = w1_awls[~w1_awls.isin([np.inf,-np.inf])]

In [ ]:
data_awls = foo.dropna().reset_index()
data_awls

In [ ]:
x_fig8a = data_awls['w1mpro']
y_fig8a = data_awls['mag_w1']
plt.figure(figsize=(9,8))
plt.axline([0,0], c='black', slope=1, linewidth=0.2)
plt.hist2d(x_fig8a,y_fig8a, bins=(50,50), norm = colors.LogNorm(), cmap = plt.cm.Greys)
plt.xlabel('AllWISE W1')
plt.ylabel('LS W1')
plt.xlim(reversed(plt.xlim(5.,19)))
plt.ylim(reversed(plt.ylim(5.,19)))

.  
.  
## Fig 8b
.  

Right panel: the number counts in the Legacy Survey catalog compared with thosefrom AllWISE, demonstrating the increased depth made possible from using The Tractor. By using optical imaging from Legacy Surveys to detect objects,photometry is measured for objects that are well below the detection limit of the AllWISE catalog.

.  
.  
## Fig 16
.  



In [ ]:
#Fig. 16
query_fig16a = ("""
SELECT r_z, g_r, z_w1, type 
FROM ls_dr6.tractor 
WHERE (type = 'EXP' or type = 'DEV' or type = 'PSF') AND
(r_z != 'inf' and r_z != 'nan') AND
(g_r != 'inf' and g_r != 'nan') AND
(z_w1 != 'inf' and z_w1 != 'nan') AND
random_id BETWEEN 11. AND 11.2
LIMIT 180000
""")
print(query_fig16a)

In [ ]:
# Fetch the W1 mag from ls_dr17.tractor and from allwise.source
df16 = qc.query(sql=query_fig16a, fmt='pandas')

#color_type = Table.read(data16, format='pandas')

df16[:10]

In [ ]:
#df16.replace([np.nan, np.inf, -np.inf], 999, inplace = True)
len(df16)

In [ ]:
df16['g_r']

In [ ]:
#x = np.linspace(0, 2 * np.pi, 400)
#y = np.sin(x ** 2) + 1.5
cond1 = (df16['type'] == 'PSF')
cond2 = (df16['type'] == 'EXP')
cond3 = (df16['type'] == 'DEV')

x00 = df16['g_r'][cond1]
y00 = df16['r_z'][cond1]
z00 = x00*y00 

fig, axs = plt.subplots(2, 3, figsize = (14, 11))
#plt.figure(figsize=(9,8))
plt.subplots_adjust(wspace = 0.18, hspace = 0.18)

#plt.hist2d(x_fig8,y_fig8, bins=(200,200), norm = colors.LogNorm(), cmap = plt.cm.Greys)
#axs[0,0].plot(x,y)
#axs[0,1].plot(x,y, 'tab:green')

#axs[0,0].hist2d(df16['g_r'][cond1],df16['r_z'][cond1], bins=400, norm = colors.LogNorm(), cmap = plt.cm.Greys)
#axs[0,0].hist2d(df16['r_z'][cond1],df16['g_r'][cond1], bins=400, norm = colors.LogNorm(), cmap = plt.cm.Greys)
h,x_edges,y_edges,im = axs[0,0].hist2d(df16['g_r'][cond1],df16['r_z'][cond1], bins=400, norm = colors.Normalize(), cmap = plt.cm.Greys)
MAX = h.max()
x_centers = 0.5*(x_edges[1:] + x_edges[:-1])
y_centers = 0.5*(y_edges[1:] + y_edges[:-1])
print(h.shape,x_centers.shape,x_edges.shape)

axs[0,0].contour(x_centers,y_centers,h,origin = 'lower', levels=np.array((0.01, 0.1, 0.4)) * MAX, linestyles = '-')
axs[0,0].set_xlim(0,2.50)
axs[0,0].set_ylim(-0.01,3)
axs[0,0].set(xlabel='r-z', ylabel='g-r')

axs[0,1].hist2d(df16['g_r'][cond2],df16['r_z'][cond2], bins=400, norm = colors.LogNorm(), cmap = plt.cm.Greys)
#axs[0,1].hist2d(df16['r_z'][cond2],df16['g_r'][cond2], bins=400, norm = colors.LogNorm(), cmap = plt.cm.Greys)
axs[0,1].set_xlim(0,2.50)
axs[0,1].set_ylim(-0.01,3)
axs[0,1].set(xlabel='r-z', ylabel='g-r')

axs[0,2].hist2d(df16['g_r'][cond3],df16['r_z'][cond3], bins=400, norm = colors.LogNorm(), cmap = plt.cm.Greys)
#axs[0,2].hist2d(df16['r_z'][cond3],df16['g_r'][cond3], bins=400, norm = colors.LogNorm(), cmap = plt.cm.Greys)
axs[0,2].set_xlim(0,2.50)
axs[0,2].set_ylim(-0.01,3)
axs[0,2].set(xlabel='r-z', ylabel='g-r')

axs[1,0].hist2d(df16['r_z'][cond1],df16['z_w1'][cond1], bins=400, norm = colors.LogNorm(), cmap = plt.cm.Greys)
#axs[1,0].hist2d(df16['z_w1'][cond1],df16['r_z'][cond1], bins=400, norm = colors.LogNorm(), cmap = plt.cm.Greys)
axs[1,0].set_xlim(0,3)
axs[1,0].set_ylim(-2,2.5)
axs[1,0].set(xlabel='r-z', ylabel='z-W1')

axs[1,1].hist2d(df16['r_z'][cond2],df16['z_w1'][cond2], bins=400, norm = colors.LogNorm(), cmap = plt.cm.Greys)
#axs[1,1].hist2d(df16['z_w1'][cond2],df16['r_z'][cond2], bins=400, norm = colors.LogNorm(), cmap = plt.cm.Greys)
axs[1,1].set_xlim(0,3)
axs[1,1].set_ylim(-2,2.5)
axs[1,1].set(xlabel='r-z', ylabel='z-W1')

axs[1,2].hist2d(df16['r_z'][cond3],df16['z_w1'][cond3], bins=400, norm = colors.LogNorm(), cmap = plt.cm.Greys)
#axs[1,2].hist2d(df16['z_w1'][cond3],df16['r_z'][cond3], bins=400, norm = colors.LogNorm(), cmap = plt.cm.Greys)
axs[1,2].set_xlim(0,3)
axs[1,2].set_ylim(-2,2.5)
axs[1,2].set(xlabel='r-z', ylabel='z-W1')

plt.show()
# Hide x labels and tick labels for top plots and y ticks for right plots.
#for ax in axs.flat:
#    ax.label_outer()
    

In [ ]:
x = np.arange(1, 10)
y = x.reshape(-1, 1)
h = x * y

cs = plt.contourf(h, levels=[10, 30, 50],
    colors=['#808080', '#A0A0A0', '#C0C0C0'], extend='both')
cs.cmap.set_over('red')
cs.cmap.set_under('blue')
cs.changed()


In [ ]:
(df16['r_z'][cond1] == 999.0).all()